In [1]:
from pymongo import MongoClient

client = MongoClient("localhost", 27017)

# Assign the database to a variable
db = client.NonRelProj 

# Take the collection "Reviews"
reviews = db.Reviews

# Take the collection "MetaData"
meta = db.MetaData

## Trust Index

Everyone knows that when looking for restaurants, the average rating alone is not sufficient. In fact, the number of reviews received by a certain restaurant is an indicator used by people to assess whether its rating is trustworthy or not.

Formula for Trustworthiness Index

trust_index = (R * N) / (N + m)

Where:

R = avg_rating of the business

N = num_of_reviews (number of reviews)

m = a weighting factor (e.g., the average number of reviews across all businesses)

This formula ensures that businesses with very few reviews start closer to the average rating, while those with more reviews gradually reflect their actual rating.

In [2]:
# Step 1: Calculate the average number of 
# reviews across all businesses
total_reviews = meta.aggregate([
    {"$group": {
        "_id": None, "avg_num_reviews": {
            "$avg": "$num_of_reviews"}}}
])

# Get the computed average number of reviews
m = list(total_reviews)[0]["avg_num_reviews"]  

# Step 2: Compute trust_index and update each document
for doc in meta.find({}, {
    "_id": 1, "avg_rating": 1, "num_of_reviews": 1}):
    # Get rating (default to 0 if missing)
    R = doc.get("avg_rating", 0)
    # Get number of reviews (default to 0)  
    N = doc.get("num_of_reviews", 0)  

    # Compute the trust_index
    trust_index = (R * N) / (N + m) if N > 0 else 0

    # Update the document with the new trust_index
    meta.update_one({"_id": doc["_id"]}, {
        "$set": {"trust_index": trust_index}})

print("Trustworthiness index added to all documents.")


Trustworthiness index added to all documents.


Reduces the impact of few reviews:
- If a restaurant has 1 review with 5 stars, its score won't immediately be 5.
- It will be closer to the average rating until more reviews confirm its quality.

Gives more weight to businesses with many reviews:
- A 4.5-star rating with 500 reviews is considered more trustworthy than 4.8 stars with 5 reviews.

Automatically adapts to the dataset:
- The m factor (average number of reviews) adjusts dynamically based on the dataset.

This formula ensures that:
- If N is large, the index is close to R (businesses with many reviews get scores that closely match their rating).
- If N is small, the index is adjusted towards a neutral value (to avoid giving small-sample ratings too much weight).

In [3]:
# My coordinates (latitude and longitude) 
# to be given as input location
my_latitude = 21.924238
my_longitude = -159.510035

# Distance in meters within which I am filtering
radius = 1500  

# Query to find restaurants 
# near you, sorted by trust_index
restaurant_close_trust = meta.aggregate([
    {
        "$geoNear": {
            "near": {
                "type": "Point", "coordinates": 
                [my_longitude, my_latitude]}, 
            # Field for distance calculation
            "distanceField": "distance", 
            # Max distance in meters 
            "maxDistance": radius,  
            # Use spherical geometry for accurate distances
            "spherical": True  
        }
    },
    {   # Sort by trust_index (descending order)
        "$sort": {"trust_index": -1}  
    },
    {
        "$project": {"_id":1,
                     "name":1,
                     "num_of_reviews":1,
                     "avg_rating":1,
                     "trust_index":1
                     }
    }
])

# Convert to list and print results
results = list(restaurant_close_trust)
for business in results:
    print(business)


{'_id': ObjectId('67c751b08c4ae1c3ac1c1eb5'), 'name': 'Kalaheo Cafe & Coffee Company', 'avg_rating': 4.7, 'num_of_reviews': 867, 'trust_index': 3.512003604783395}
{'_id': ObjectId('67c751af8c4ae1c3ac1c1853'), 'name': 'Brick Oven Pizza', 'avg_rating': 4.3, 'num_of_reviews': 444, 'trust_index': 2.5895257722949645}
{'_id': ObjectId('67c751b08c4ae1c3ac1c22cf'), 'name': 'The Fresh Shave', 'avg_rating': 4.8, 'num_of_reviews': 254, 'trust_index': 2.2277532240104776}
{'_id': ObjectId('67c751ad8c4ae1c3ac1be454'), 'name': 'Kauai Poke Co.Food Truck', 'avg_rating': 4.8, 'num_of_reviews': 58, 'trust_index': 0.7925350207487906}
{'_id': ObjectId('67c751ad8c4ae1c3ac1bec13'), 'name': 'Uncle Bobby’s', 'avg_rating': 5, 'num_of_reviews': 14, 'trust_index': 0.22780685124190772}
